In [ ]:
# !pip install openai

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

WEAVIATE_HTTP_URL = os.getenv("WEAVIATE_HTTP_URL")
WEAVIATE_GRPC_URL = os.getenv("WEAVIATE_GRPC_URL")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_BASE_URL = os.getenv("AZURE_BASE_URL")
AZURE_RESOURCE_NAME = os.getenv("AZURE_RESOURCE_NAME")
USERNAME = os.getenv("USERNAME")
WIKI_NAME = f"{USERNAME}_wiki"

print(WIKI_NAME)

## Generate query from prompt

Set up OpenAI client

### Option A - with OpenAI Azure API

In [ ]:
from openai import AzureOpenAI

# point to cert chain
os.environ['SSL_CERT_FILE'] = 'ca-bundle-full.crt'
# Notice the AZURE_OPENAI_API_KEY environment variable is required
openai_client = AzureOpenAI(
    azure_endpoint=AZURE_BASE_URL,
    api_version=AZURE_RESOURCE_NAME, # "2024-07-01-preview",
    api_key=AZURE_OPENAI_API_KEY
)

### Option B - with OpenAI API

In [ ]:
# from openai import OpenAI

# # Directly with OpenAI
# OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# # from openai import OpenAI
# openai_client = OpenAI(
#     api_key=OPENAI_API_KEY,
#     base_url="https://api.openai.com/v1"
# )

### Prepare `generate_query_from_promt` function

In [ ]:
def generate_query_from_promt(prompt):
    response = openai_client.chat.completions.create(
        deployment_id="gpt-4o-mini-0718",
        messages=[
            { 
                "role": "system",
                "content": "Your job is to extract a query from the provided user prompt, the query will then be used to run a vector search in a vector database." },
            { 
                "role": "user",
                "content": f"Please give me a 2-3 word query that can be used to find relevant info to the following prompt - {prompt}"
            },
        ]
    )
    return response.choices[0].message.content

In [ ]:
# Example of how to generate a query from a prompt
generate_query_from_promt("Where do the tallest penguins live?")

## Connect to Weaviate

In [ ]:
import weaviate
from weaviate.classes.init import AdditionalConfig, Timeout

client = weaviate.connect_to_custom(
    http_host=WEAVIATE_HTTP_URL,
    http_port=80,
    http_secure=False,
    grpc_host=WEAVIATE_GRPC_URL,
    grpc_port=50051,
    grpc_secure=False,

    headers = {
        "X-Azure-Api-Key": AZURE_OPENAI_API_KEY
    },

    # LLMs are slower than embedding models
    # we need to increase the timeout
    additional_config=AdditionalConfig(
        timeout=Timeout(query=60),  # Values in seconds
    )
)

client.is_ready()

## Two-step RAG

In [ ]:
BLUE   = "\033[94m"
PURPLE = "\033[95m"
RESET  = "\033[0"

def two_step_rag(user_prompt):
    # Step 1
    prompt = user_prompt + " Please only use the provided content with this prompt. Don't make things up."
    
    generated_query = generate_query_from_promt(prompt)
    print("=== Generated Query ===")
    print(f"Generated query: {generated_query}")

    # Step 2
    wiki = client.collections.get(WIKI_NAME)

    response = wiki.generate.near_text(
        query=generated_query,
        auto_limit=1,

        grouped_task=prompt,
        grouped_properties=["text", "title"]
    )

    # Print results
    print(f"\n{BLUE}=== Generated Response ===")
    print(response.generative.text)

    print(f"\n{PURPLE}=== Source ===")
    for item in response.objects:
        print(item.properties)

In [ ]:
two_step_rag("How do airplanes fly? Help me write a 3 paragraph assignment for my school project. Oh btw, I am 10 years old.")

In [ ]:
two_step_rag("What are the pros and cons of automation using computer?")

In [ ]:
two_step_rag("How do CPUs work?")

## Close the client

In [ ]:
client.close()